In [6]:
from selenium import webdriver
from msedge.selenium_tools import options as EdgeOptions
from msedge.selenium_tools import Edge, EdgeOptions
import time 

def url(url, driver_path):
    options = EdgeOptions()
    options.add_argument("--lang=vi")

    # Create a Microsoft Edge WebDriver instance with options
    driver = Edge(executable_path=driver_path, options=options)

    driver.get(url)

    # Add your code here to perform actions on the webpage

    time.sleep(2)

    return driver   
    
# Set the desired URL and driver path
desired_url = "https://www.google.com/maps/place/B%E1%BB%87nh+vi%E1%BB%87n+Nhi+%C4%90%E1%BB%93ng+2/@10.7808477,106.6997441,17z/data=!4m8!3m7!1s0x31752f49a8134407:0x8b3ac844e0a002a4!8m2!3d10.7808424!4d106.702319!9m1!1b1!16s%2Fg%2F1v8w_p4f?entry=ttu"
desired_driver_path = 'C:\Program Files (x86)\Microsoft\Edge\Application\msedgedriver.exe'

# Call the function with the desired URL and driver path
driver = url(desired_url, desired_driver_path)

C:\Users\NguyenDucHuy\AppData\Local\Temp\ipykernel_23292\3157166033.py:11: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(executable_path=driver_path, options=options)


In [7]:
print(driver.title)

Children's Hospital 2 - Google Maps


In [8]:
total_number_of_reviews = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[2]/div/div[2]/div[2]').text.split(" ")[0]
print(total_number_of_reviews)
total_number_of_reviews = int(total_number_of_reviews.replace(',','')) if ',' in total_number_of_reviews else int(total_number_of_reviews)

1,446


In [9]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

SCROLL_PAUSE_TIME = 2
MAX_WAIT_TIME_SECONDS = 100
TARGET_ELEMENT_COUNT = 900

scrollable_div = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]')

# Get scroll height
last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

start_time = time.time()
while True:
    # Scroll down to bottom
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Check if the desired number of elements is reached
    elements = driver.find_elements_by_css_selector('.wiI7pd')
    if len(elements) >= TARGET_ELEMENT_COUNT:
        print(f"Desired element count ({TARGET_ELEMENT_COUNT}) reached.")
        break

    # Check if the script has been running for more than 10 seconds
    # if time.time() - start_time > MAX_WAIT_TIME_SECONDS:
    #    print("Timeout: Script running for more than 100 seconds.")
    #    break

Desired element count (900) reached.


In [10]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Xác định các nút "See more"
see_more_buttons = driver.find_elements(By.CSS_SELECTOR, '[aria-label="See more"]')
clicked_see_more = False  # Biến đánh dấu đã nhấp vào nút "See more" hay chưa

while see_more_buttons or not clicked_see_more:
    clicked_see_more = False  # Đặt lại giá trị của biến đánh dấu
    for button in see_more_buttons:
        actions = ActionChains(driver)
        actions.move_to_element(button).perform()
        try:
            # Sử dụng WebDriverWait để chờ lâu hơn và tìm element theo CSS_SELECTOR
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[aria-label="See more"]'))).click()
            clicked_see_more = True  # Đánh dấu đã nhấp vào nút "See more"
            time.sleep(2)
        except Exception as e:
            # Ghi log lỗi ra để kiểm tra
            print("Error occurred while clicking 'See more' button:", str(e))

    # Cập nhật danh sách nút "See more" sau khi nhấp vào
    see_more_buttons = driver.find_elements(By.CSS_SELECTOR, '[aria-label="See more"]')

Error occurred while clicking 'See more' button: Message: stale element reference: stale element not found
  (Session info: MicrosoftEdge=114.0.1823.79)

Error occurred while clicking 'See more' button: 'NoneType' object has no attribute 'is_displayed'


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=114.0.1823.79)


In [ ]:
from bs4 import BeautifulSoup

def get_review_summary(result_set):
    rev_dict = {
        'Review Time': [],
        'Review Text' : [],
        'Review Rate': [],}
    for result in result_set:

        review_rate = result.find('span', class_='kvMYJc')["aria-label"]
        review_time = result.find('span', class_="rsqaWe").text.strip()
        review_text = result.find('span',class_='wiI7pd')
    
        if review_text is None:
            rev_dict['Review Text'].append('')
        else:
            rev_dict['Review Text'].append(review_text.text.strip())
        
        rev_dict["Review Time"].append(review_time)
        
        rev_dict['Review Rate'].append(review_rate)
    return rev_dict

In [ ]:
import pandas as pd
response = BeautifulSoup(driver.page_source, 'html.parser')
reviews = response.find_all('div', class_='jftiEf')
rev_dict = get_review_summary(reviews)

In [ ]:
print(len(rev_dict["Review Text"]))

In [ ]:
df = pd.DataFrame.from_dict(rev_dict)
df.columns = ["Time", "Review Text", "Rating"]

In [ ]:
df.to_csv("BVNHIDONG2-review.csv", encoding='utf-8-sig')

In [ ]:
pd.read_csv("BVNHIDONG2-review.csv", index_col="Unnamed: 0")

In [ ]:
df["Review Text"][1139]

In [ ]:
df.info()

In [ ]:
# Count the occurrences of each number in the "review_rating" column
rating_counts = df['Rating'].value_counts().sort_index()

# Display the counts
for rating, count in rating_counts.items():
    print(f"Rating {rating}: {count} occurrences")

In [ ]:
import pandas as pd

# Read the CSV file containing your dataset
# df = pd.read_csv('dataset.csv')

# Define a function to assign sentiment labels based on the "Rating" column
def assign_sentiment(Rating):
    if Rating in ['1 star', '2 stars']:
        return "Negative"
    elif Rating == '3 stars':
        return "Neutral"
    elif Rating in ['4 stars', '5 stars']:
        return "Positive"
    else:
        return "Unknown"

# Apply the function to create the "sentiment" column
df['sentiment'] = df['Rating'].apply(assign_sentiment)
 
# Save the updated DataFrame to a new CSV file
df.to_csv('BVNHIDONG2.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd 
data = pd.read_csv('BVNHIDONG2.csv', encoding='utf-8')

In [ ]:
# Count the occurrences of each number in the "review_rating" column
rating_counts = data['sentiment'].value_counts().sort_index()

# Display the counts
for rating, count in rating_counts.items():
    print(f"Rating {rating}: {count} occurrences")

In [ ]:
import matplotlib.pyplot as plt

# Define colors for each rating
colors = ['red', 'orange', 'yellow', 'green', 'blue']

# Create a bar chart with colors
plt.bar(rating_counts.index, rating_counts.values, color=colors)

# Set labels and title
plt.xlabel('Rating')
plt.ylabel('Count')
plt.title('Count of Ratings')

# Display the chart
plt.show()

In [ ]:
data.info()